# Magnetic System Explorer

Comprehensive notebook for magnetic system analysis with real-time visualization.
Supports any magnetic material with interactive simulation and phase diagram generation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from spinlab.analysis.visualization import SpinVisualizer
from spinlab.core.spin_system import SpinSystem
from spinlab.core.hamiltonian import Hamiltonian
from ase.io import read
from ase.build import bulk, make_supercell
import time

# Configure matplotlib for notebook
%matplotlib inline

# Initialize visualizer
viz = SpinVisualizer()
print("Magnetic System Explorer Ready!")

Magnetic System Explorer Ready!


In [ ]:
# System parameters - MODIFY THESE FOR YOUR ANALYSIS
##################

# Structure parameters
material = 'Fe'               # Material symbol for ASE
crystal = 'bcc'               # Crystal structure
lattice_param = 2.87          # Lattice parameter (Å)
n_cells = 8                   # Supercell size (8x8x1)
system_size = (n_cells, n_cells, 1)

# Create ASE structure
structure = bulk(material, crystal, a=lattice_param, cubic=True)
structure = structure.repeat(system_size)

print(f"Created structure: {len(structure)} atoms ({material} {crystal})")

# Exchange interactions (eV)
j1 = -0.02                    # 1st neighbor exchange
j2 = 0.0                      # 2nd neighbor exchange
j3 = 0.0                      # 3rd neighbor exchange

# Anisotropic exchange (eV) - for 1st neighbors only
jxx = 0.0; jyy = 0.0; jzz = 0.0
jxy = 0.0; jxz = 0.0; jyz = 0.0

# Single-ion anisotropy (eV)
anisotropy = 0.0              # Easy-axis anisotropy

# External fields
g_factor = 2.0                # g-factor
gamma = 0.0                   # Electric coupling (e·Å)
b_field = 0.0                 # Magnetic field (Tesla)
e_field = 0.0                 # Electric field (V/Å)

# Neighbor cutoffs (Å) - you still need to define these
cutoff_distance = 3.5         # 1st neighbor cutoff
second_cutoff = 4.9           # 2nd neighbor cutoff  
third_cutoff = 6.1            # 3rd neighbor cutoff

# Simulation parameters
temperature = 100             # Temperature (K)
spin_magnitude = 1.0          # Spin magnitude
model_type = '3D'             # 'Ising', 'XY', '3D'
angular_res = 1.0             # Angular resolution (degrees)

# Monte Carlo parameters
mc_steps = 1000
update_interval = 50

print(f"Parameters set: j1={j1}, T={temperature}K, Model={model_type}")

In [ ]:
# Create comprehensive Hamiltonian
##################

# Optional: Define non-magnetic species for multi-component systems
# nonmagnetic_species = ['O', 'H']  # Remove these atoms from magnetic simulation
nonmagnetic_species = None  # None = all atoms are magnetic (intuitive!)

hamiltonian = Hamiltonian(nonmagnetic_species=nonmagnetic_species)

# Filter structure if non-magnetic species are specified
if nonmagnetic_species is not None:
    structure, index_map = hamiltonian.filter_magnetic_atoms(structure)
    print(f"Removed non-magnetic species: {nonmagnetic_species}")
else:
    index_map = None
    print("All atoms considered magnetic")

# Add exchange interactions for multiple shells
if j1 != 0.0:
    hamiltonian.add_exchange(J=j1, neighbor_shell="shell_1", name="j1_exchange")
if j2 != 0.0:
    hamiltonian.add_exchange(J=j2, neighbor_shell="shell_2", name="j2_exchange")
if j3 != 0.0:
    hamiltonian.add_exchange(J=j3, neighbor_shell="shell_3", name="j3_exchange")

# Add anisotropic exchange (using Kitaev terms for directional coupling)
if any([jxx, jyy, jzz]) != 0.0:
    hamiltonian.add_kitaev(
        K_couplings={"x": jxx, "y": jyy, "z": jzz},
        neighbor_shell="shell_1",
        name="anisotropic_exchange"
    )

# Add single-ion anisotropy
if anisotropy != 0.0:
    hamiltonian.add_single_ion_anisotropy(K=anisotropy, axis=[0, 0, 1], name="easy_axis")

# Add magnetic field (Zeeman term)
if b_field != 0.0:
    hamiltonian.add_magnetic_field(B_field=[0, 0, b_field], g_factor=g_factor, name="zeeman")

# Add electric field coupling
if e_field != 0.0:
    hamiltonian.add_electric_field(E_field=[0, 0, e_field], gamma=gamma, name="electric")

print("Hamiltonian terms added:")
for i, name in enumerate(hamiltonian.term_names):
    print(f"  {i+1}. {name}")

In [ ]:
# Create SpinSystem
##################
system = SpinSystem(
    structure=structure,
    hamiltonian=hamiltonian,
    magnetic_model=model_type.lower(),
    spin_magnitude=spin_magnitude
)

# Setup neighbor shells
##################
# Note: You still need to define cutoff distances manually
# SpinLab automatically creates "shell_1", "shell_2", etc. based on cutoff order
cutoffs = [cutoff_distance]
if j2 != 0.0:
    cutoffs.append(second_cutoff)
if j3 != 0.0:
    cutoffs.append(third_cutoff)

neighbors = system.get_neighbors(cutoffs)

print("Neighbor shells setup:")
for shell, neighbor_array in neighbors.items():
    avg_neighbors = np.mean(np.sum(neighbor_array >= 0, axis=1))
    print(f"  {shell}: {neighbor_array.shape} (avg {avg_neighbors:.1f} neighbors/site)")

# Initialize spin configuration based on model type
##################
if model_type.lower() == 'ising':
    # Ising: ±Z spins only - need to implement this method
    print("Ising model: initializing ±Z spins")
    # For now, use random and constrain to ±Z
    system.random_configuration()
    # Constrain to ±Z directions
    system.spin_config[:, :2] = 0  # Zero x,y components
    system.spin_config[:, 2] = np.sign(system.spin_config[:, 2]) * spin_magnitude
elif model_type.lower() == 'xy':
    # XY: spins in xy-plane
    print("XY model: initializing xy-plane spins")
    system.random_configuration()
    # Constrain to xy-plane
    system.spin_config[:, 2] = 0  # Zero z component
    # Normalize xy components
    xy_norm = np.linalg.norm(system.spin_config[:, :2], axis=1, keepdims=True)
    system.spin_config[:, :2] = system.spin_config[:, :2] / xy_norm * spin_magnitude
else:
    # 3D: random 3D configuration
    print("3D model: initializing random 3D spins")
    system.random_configuration()

# System information
##################
initial_energy = system.calculate_energy()
initial_magnetization = system.calculate_magnetization()

print(f"\nSystem created: {len(system.positions)} magnetic sites")
print(f"Initial energy: {initial_energy:.4f} eV")
print(f"Initial |M|: {np.linalg.norm(initial_magnetization):.3f}")
print(f"Spin configuration shape: {system.spin_config.shape}")
print(f"Energy per site: {initial_energy/len(system.positions):.6f} eV")

## Initial Spin Configuration

Visualize the random initial spin state:

In [ ]:
# Plot initial random configuration using actual positions
viz.plot_spin_configuration(
    system.positions,
    system.spin_config,
    title=f"{material} Initial Random Configuration",
    figsize=(8, 8)
)

## Real-time Monte Carlo Simulation

Monitor the simulation progress with live energy and magnetization tracking:

In [ ]:
# Real-time Monte Carlo simulation with progress tracking
##################

from spinlab.simulation.monte_carlo import MonteCarloSimulator
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Initialize Monte Carlo simulator
mc_sim = MonteCarloSimulator(system, temperature)

# Storage for tracking simulation progress
energy_history = []
magnetization_history = []
time_history = []

# Real-time simulation function
def run_realtime_simulation(steps, update_interval=50):
    """Run Monte Carlo with real-time progress visualization."""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    for step in range(steps):
        # Perform Monte Carlo step
        mc_sim.single_sweep()
        
        # Track properties every few steps
        if step % 10 == 0:
            energy = system.calculate_energy()
            magnetization = np.linalg.norm(system.calculate_magnetization())
            
            energy_history.append(energy)
            magnetization_history.append(magnetization)
            time_history.append(step)
        
        # Update visualization every update_interval steps
        if step % update_interval == 0 and step > 0:
            clear_output(wait=True)
            
            # Plot 1: Current spin configuration
            ax1.clear()
            viz.plot_spin_configuration(
                system.positions,
                system.spin_config,
                title=f"Step {step}: {material} Spin Configuration",
                ax=ax1
            )
            
            # Plot 2: Energy evolution
            ax2.clear()
            ax2.plot(time_history, energy_history, 'b-', linewidth=2)
            ax2.set_xlabel('MC Steps')
            ax2.set_ylabel('Energy (eV)')
            ax2.set_title('Energy Evolution')
            ax2.grid(True, alpha=0.3)
            
            # Plot 3: Magnetization evolution
            ax3.clear()
            ax3.plot(time_history, magnetization_history, 'r-', linewidth=2)
            ax3.set_xlabel('MC Steps')
            ax3.set_ylabel('|Magnetization|')
            ax3.set_title('Magnetization Evolution')
            ax3.grid(True, alpha=0.3)
            
            # Plot 4: Phase space trajectory (Energy vs Magnetization)
            ax4.clear()
            ax4.scatter(energy_history, magnetization_history, c=time_history, 
                       cmap='viridis', s=20, alpha=0.7)
            ax4.set_xlabel('Energy (eV)')
            ax4.set_ylabel('|Magnetization|')
            ax4.set_title('Phase Space Trajectory')
            ax4.grid(True, alpha=0.3)
            
            plt.tight_layout()
            display(fig)
            
            # Print current status
            current_energy = energy_history[-1] if energy_history else 0
            current_mag = magnetization_history[-1] if magnetization_history else 0
            print(f"Step {step}: E = {current_energy:.4f} eV, |M| = {current_mag:.3f}")
    
    return energy_history, magnetization_history, time_history

print("Ready to run real-time simulation!")
print(f"Will run {mc_steps} steps with updates every {update_interval} steps")

In [ ]:
# Run the real-time simulation
##################
print("Starting real-time Monte Carlo simulation...")
print("Watch the spin configuration evolve and track thermodynamic properties!")

# Run simulation with real-time monitoring
final_energy, final_mag, final_time = run_realtime_simulation(
    mc_steps, 
    update_interval=update_interval
)

print(f"\nSimulation completed!")
print(f"Final energy: {final_energy[-1]:.4f} eV")
print(f"Final |magnetization|: {final_mag[-1]:.3f}")
print(f"Total steps: {final_time[-1]}")

## Temperature Sweep Analysis

Perform comprehensive thermodynamic analysis across temperature range:

In [ ]:
# Temperature sweep for thermodynamic analysis
##################

def temperature_sweep_analysis(t_min=10, t_max=500, n_temps=20, equilibration_steps=500, sampling_steps=1000):
    """
    Comprehensive temperature sweep analysis matching SpinMCPack workflow.
    
    Args:
        t_min, t_max: Temperature range (K)
        n_temps: Number of temperature points
        equilibration_steps: Steps for equilibration at each temperature
        sampling_steps: Steps for property sampling
    """
    
    # Temperature range
    temperatures = np.linspace(t_min, t_max, n_temps)
    
    # Storage arrays
    avg_energy = np.zeros(n_temps)
    avg_magnetization = np.zeros(n_temps)
    heat_capacity = np.zeros(n_temps)
    magnetic_susceptibility = np.zeros(n_temps)
    
    # Simulation results tracking
    all_energies = []
    all_magnetizations = []
    
    print(f"Temperature sweep: {t_min}K → {t_max}K ({n_temps} points)")
    print("This replicates the comprehensive SpinMCPack workflow...")
    
    for i, temp in enumerate(temperatures):
        print(f"\nTemperature {i+1}/{n_temps}: {temp:.1f}K")
        
        # Update Monte Carlo temperature
        mc_sim.temperature = temp
        
        # Equilibration phase
        print(f"  Equilibrating for {equilibration_steps} steps...")
        for _ in range(equilibration_steps):
            mc_sim.single_sweep()
        
        # Sampling phase
        energies = []
        magnetizations = []
        
        print(f"  Sampling for {sampling_steps} steps...")
        for step in range(sampling_steps):
            mc_sim.single_sweep()
            
            # Sample properties every 10 steps
            if step % 10 == 0:
                energy = system.calculate_energy()
                mag_vector = system.calculate_magnetization()
                mag_magnitude = np.linalg.norm(mag_vector)
                
                energies.append(energy)
                magnetizations.append(mag_magnitude)
        
        # Calculate thermodynamic properties
        energies = np.array(energies)
        magnetizations = np.array(magnetizations)
        
        # Averages
        avg_energy[i] = np.mean(energies)
        avg_magnetization[i] = np.mean(magnetizations)
        
        # Fluctuations for response functions
        energy_fluctuation = np.var(energies)
        mag_fluctuation = np.var(magnetizations)
        
        # Heat capacity: C = (⟨E²⟩ - ⟨E⟩²) / (k_B T²)
        # Using k_B in eV/K: 8.617e-5
        k_B = 8.617e-5  # eV/K
        heat_capacity[i] = energy_fluctuation / (k_B * temp**2)
        
        # Magnetic susceptibility: χ = (⟨M²⟩ - ⟨M⟩²) / (k_B T)
        magnetic_susceptibility[i] = mag_fluctuation / (k_B * temp)
        
        # Store full data
        all_energies.append(energies)
        all_magnetizations.append(magnetizations)
        
        print(f"  ⟨E⟩ = {avg_energy[i]:.4f} eV, ⟨|M|⟩ = {avg_magnetization[i]:.3f}")
        print(f"  C = {heat_capacity[i]:.2e}, χ = {magnetic_susceptibility[i]:.2e}")
    
    return {
        'temperatures': temperatures,
        'avg_energy': avg_energy,
        'avg_magnetization': avg_magnetization, 
        'heat_capacity': heat_capacity,
        'magnetic_susceptibility': magnetic_susceptibility,
        'all_energies': all_energies,
        'all_magnetizations': all_magnetizations
    }

print("Temperature sweep analysis function ready!")
print("This will replicate the comprehensive SpinMCPack thermodynamic analysis")

In [ ]:
# Run temperature sweep (uncomment to execute)
##################

# CAUTION: This may take significant time depending on parameters
# Adjust temperature range and steps for your system

run_temperature_sweep = False  # Set to True to run

if run_temperature_sweep:
    print("Running comprehensive temperature sweep...")
    
    # Quick sweep for demonstration
    results = temperature_sweep_analysis(
        t_min=50,          # Start temperature (K)
        t_max=300,         # End temperature (K)  
        n_temps=10,        # Number of temperature points
        equilibration_steps=200,  # Equilibration steps per temperature
        sampling_steps=500        # Sampling steps per temperature
    )
    
    # Store results for plotting
    temp_sweep_results = results
    print("Temperature sweep completed!")
    
else:
    print("Temperature sweep not executed (set run_temperature_sweep=True to run)")
    print("This will generate comprehensive thermodynamic data like SpinMCPack")

## Comprehensive Analysis and Visualization

Create publication-quality plots of thermodynamic properties: